# Step 1.1.1 Stage Libraries
Libraries such as [AutoGluon](https://auto.gluon.ai) are not part of the default Snowpark/Anaconda distributions. Snowflake
does allow us to use 3rd party libraries in our UDF/UDTF as documented here:    
- Doc: [Creating a Python UDF With Code Uploaded from a Stage](https://docs.snowflake.com/en/developer-guide/udf/python/udf-python-creating.html#creating-a-python-udf-with-code-uploaded-from-a-stage)
- Doc: [Using Third-Party Packages](https://docs.snowflake.com/en/developer-guide/udf/python/udf-python-packages.html#using-third-party-packages)

Hence we need to 
  - download libraries 
  - load the libraries to an internal stage

In [31]:
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
# from snowflake.snowpark.types import * 
# from snowflake.snowpark.functions import *
import configparser

PROJECT_HOME_DIR = '../../..'
CONFIG_FL = f'{PROJECT_HOME_DIR}/config.ini'

# Source various helper functions
%run ./scripts/notebook_helpers.py


In [32]:
# Initialization

set_cell_background('#EAE3D2')

config = configparser.ConfigParser()
sflk_session = None

print(" Initialize Snowpark session")
with open(CONFIG_FL) as f:
    config.read(CONFIG_FL)
    snow_conn_flpath =  f"{PROJECT_HOME_DIR}/{config['DEFAULT']['connection_fl']}"
    
    # ------------
    # Connect to snowflake
    with open(snow_conn_flpath) as conn_f:
        snow_conn_info = json.load(conn_f)
        sflk_session = Session.builder.configs(snow_conn_info).create()

if(sflk_session == None):
    raise(f'Unable to connect to snowflake. Validate connection information in file: {CONFIG_FL} ')

df = sflk_session.sql('select current_warehouse(), current_user(), current_role();').to_pandas()
display(df)


 Initialize Snowpark session


,CURRENT_WAREHOUSE(),CURRENT_USER(),CURRENT_ROLE()
0,LAB_WH,VSEKAR,DEV_BLOGGER


---
## The Libraries

We are downloading the various Wheel files directly from PyPi. These URLs are not same and will change with different versions. 
It is very highly possible, that there is a newer version of libraries when you are running this experiement. Should you prefer
to use the latest, then you would need to update the urls.

If you notice below, I am purposefully downloading specific modules of AutoGluon (ex: autogluon-tabular). The reason is that the
base library (autogluon-XXX) does not have any of the modules by default. When you do a Pip install, then all these sub-modules
seems to get downloaded. Now in a managed secure environment like where the python UDF would be running, no network connection 
is possible, as the networks calls are blocked. Hence we are taking the effort of downloading these sub-modules ourselves.

The libraries would be downloaded to a local directory 'temp/lib', which will get created as part of this execution.

In [33]:
# Download libraries
set_cell_background('#EAE3D2')

LOCAL_TEMP_DIR = os.path.join(PROJECT_HOME_DIR, 'temp') 
LOCAL_LIB_DIR = os.path.join(LOCAL_TEMP_DIR, 'libs')
LIB_URLS = [
    'https://files.pythonhosted.org/packages/a8/51/f5a35cc8357b3c4d1da50f535335026741a21d375baf4839c382f3c9a609/autogluon.tabular-0.5.2-py3-none-any.whl'
    ,'https://files.pythonhosted.org/packages/fc/79/9aeb3317bfe7339f077eb43bc0bafdeaa216c33be0263993adad707e5cab/autogluon.core-0.5.2-py3-none-any.whl'
    ,'https://files.pythonhosted.org/packages/32/e0/aa06472d4f526fa827c7f543939c4bf53d55ee7897281984d369f7bd63a6/autogluon.common-0.5.2-py3-none-any.whl'
    ,'https://files.pythonhosted.org/packages/64/88/3e24833d2e8798f49e68ba392d88eb8c59c649d020117c593e551f17608e/autogluon.extra-0.3.1-py3-none-any.whl'
    ,'https://files.pythonhosted.org/packages/00/5a/e57d5cfb4a88a9a80ea2eecd2354a6cdd61846714c277323d9c03e585b51/autogluon.features-0.5.2-py3-none-any.whl'
    ,'https://files.pythonhosted.org/packages/50/70/1fc8354a51f3f967f8bf6c392937b8cba0b693eda7383e2cde1af9ed636c/autogluon.multimodal-0.5.2-py3-none-any.whl'
    ,'https://files.pythonhosted.org/packages/d0/1c/dda16dd9812a22ab8b2066a7b31ced1685595825d4f294f9b27a7925a4e2/autogluon-0.5.2-py3-none-any.whl'
]

for lib_url in LIB_URLS:
    # get the file name, from the url
    splits = lib_url.split('/')
    tot_splits = len(splits)
    target_file = splits[tot_splits - 1]

    download_library_frompypi_tolocal(lib_url ,target_file ,LOCAL_LIB_DIR)

  Create local dir: ../../../temp/libs
  Create local dir: ../../../temp/libs
  Create local dir: ../../../temp/libs
  Create local dir: ../../../temp/libs
  Create local dir: ../../../temp/libs
  Create local dir: ../../../temp/libs
  Create local dir: ../../../temp/libs


Once downloaded the libraries would then be uploaded to the stage

In [36]:
# Upload libraries to stage
set_cell_background('#EAE3D2')

target_db = config['DEFAULT']['db']
target_schema = config['DEFAULT']['sch']
stage = config['DEFAULT']['stage']
stage_lib_dir = config['DEFAULT']['stage_lib_dir']

upload_locallibraries_to_p_stage(sflk_session ,LOCAL_LIB_DIR ,target_db ,target_schema ,stage ,stage_lib_dir)

 Uploading library to stage: sflk_autogluon_db.public.lib_data_stg 
    ../../../temp/libs/autogluon.multimodal-0.5.2-py3-none-any.whl => @lib_data_stg/libs/
    ../../../temp/libs/autogluon.common-0.5.2-py3-none-any.whl => @lib_data_stg/libs/
    ../../../temp/libs/autogluon.extra-0.3.1-py3-none-any.whl => @lib_data_stg/libs/
    ../../../temp/libs/autogluon-0.5.2-py3-none-any.whl => @lib_data_stg/libs/
    ../../../temp/libs/autogluon.tabular-0.5.2-py3-none-any.whl => @lib_data_stg/libs/
    ../../../temp/libs/autogluon.features-0.5.2-py3-none-any.whl => @lib_data_stg/libs/
    ../../../temp/libs/autogluon.core-0.5.2-py3-none-any.whl => @lib_data_stg/libs/


In [37]:
# List stage out
set_cell_background('#EAE3D2')

print(' List stage directory !!')
sflk_session.sql(f'alter stage {target_db}.{target_schema}.{stage} refresh; ').collect()

df = sflk_session.sql(f'select RELATIVE_PATH from directory(@{target_db}.{target_schema}.{stage}); ').to_pandas()
display(df)

 List stage directory !!


,RELATIVE_PATH
0,libs/autogluon-0.5.2-py3-none-any.whl
1,libs/autogluon.common-0.5.2-py3-none-any.whl
2,libs/autogluon.core-0.5.2-py3-none-any.whl
3,libs/autogluon.extra-0.3.1-py3-none-any.whl
4,libs/autogluon.features-0.5.2-py3-none-any.whl
5,libs/autogluon.multimodal-0.5.2-py3-none-any.whl
6,libs/autogluon.tabular-0.5.2-py3-none-any.whl


---
### Close out

    With that we are finished this section of the demo setup

In [13]:
sflk_session.close()
print('Finished!!!')

Finished!!!
